# Menginstal Java 8
Hadoop adalah framework pengolahan data yang berbasis pemrograman java

OpenJDK adalah lingkungan pengembangan untuk membangun aplikasi, applet, dan komponen menggunakan bahasa pemrograman Java.

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!java -version

!update-alternatives --set java /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java
!update-alternatives --set javac /usr/lib/jvm/java-8-openjdk-amd64/bin/javac
!update-alternatives --set jps /usr/lib/jvm/java-8-openjdk-amd64/bin/jps
!java -version

#Finding the default Java path
!readlink -f /usr/bin/java | sed "s:bin/java::"
!apt-get install openssh-server -qq > /dev/null
!service ssh start

!grep Port /etc/ssh/sshd_config

#Creating a new rsa key pair with empty password
!ssh-keygen -t rsa -P "" -f ~/.ssh/id_rsa

# See id_rsa.pub content
!more /root/.ssh/id_rsa.pub

#Copying the key to autorized keys
!cat $HOME/.ssh/id_rsa.pub > $HOME/.ssh/authorized_keys
#Changing the permissions on the key
!chmod 0600 ~/.ssh/authorized_keys

#Conneting with the local machine
!ssh -o StrictHostKeyChecking=no localhost uptime


#Downloading Hadoop 3.2.3
!wget -q https://archive.apache.org/dist/hadoop/common/hadoop-3.2.3/hadoop-3.2.3.tar.gz

#Untarring the file
!sudo tar -xzf hadoop-3.2.3.tar.gz
#Removing the tar file
!rm hadoop-3.2.3.tar.gz


#Copying the hadoop files to user/local
!cp -r hadoop-3.2.3/ /usr/local/
#-r copy directories recursively

#Adding JAVA_HOME directory to hadoop-env.sh file
!sed -i '/export JAVA_HOME=/a export JAVA_HOME=\/usr\/lib\/jvm\/java-8-openjdk-amd64' /usr/local/hadoop-3.2.3/etc/hadoop/hadoop-env.sh

import os
#Creating environment variables
#Creating Hadoop home variable

os.environ["HADOOP_HOME"] = "/usr/local/hadoop-3.2.3"
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["JRE_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64/jre"
os.environ["PATH"] += f'{os.environ["JAVA_HOME"]}/bin:{os.environ["JRE_HOME"]}/bin:{os.environ["HADOOP_HOME"]}/sbin'


openjdk version "11.0.22" 2024-01-16
OpenJDK Runtime Environment (build 11.0.22+7-post-Ubuntu-0ubuntu222.04.1)
OpenJDK 64-Bit Server VM (build 11.0.22+7-post-Ubuntu-0ubuntu222.04.1, mixed mode, sharing)
update-alternatives: using /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java to provide /usr/bin/java (java) in manual mode
update-alternatives: using /usr/lib/jvm/java-8-openjdk-amd64/bin/javac to provide /usr/bin/javac (javac) in manual mode
update-alternatives: using /usr/lib/jvm/java-8-openjdk-amd64/bin/jps to provide /usr/bin/jps (jps) in manual mode
openjdk version "1.8.0_402"
OpenJDK Runtime Environment (build 1.8.0_402-8u402-ga-2ubuntu1~22.04-b06)
OpenJDK 64-Bit Server VM (build 25.402-b06, mixed mode)
/usr/lib/jvm/java-8-openjdk-amd64/jre/
 * Starting OpenBSD Secure Shell server sshd
   ...done.
#Port 22
#GatewayPorts no
Generating public/private rsa key pair.
Created directory '/root/.ssh'.
Your identification has been saved in /root/.ssh/id_rsa
Your public key has been saved in 

In [2]:
#Adding required property to core-site.xlm file
%%bash
cat <<EOF > $HADOOP_HOME/etc/hadoop/core-site.xml
<?xml version="1.0" encoding="UTF-8"?>
<?xml-stylesheet type="text/xsl" href="configuration.xsl"?>
<!--
  Licensed under the Apache License, Version 2.0 (the "License");
  you may not use this file except in compliance with the License.
  You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

  Unless required by applicable law or agreed to in writing, software
  distributed under the License is distributed on an "AS IS" BASIS,
  WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
  See the License for the specific language governing permissions and
  limitations under the License. See accompanying LICENSE file.
-->

<!-- Put site-specific property overrides in this file. -->

<configuration>
  <property>
          <name>fs.defaultFS</name>
          <value>hdfs://localhost:9000</value>
          <description>Where HDFS NameNode can be found on the network</description>
  </property>
</configuration>
EOF

In [3]:
%%bash
cat <<EOF > $HADOOP_HOME/etc/hadoop/hdfs-site.xml
<?xml version="1.0" encoding="UTF-8"?>
<?xml-stylesheet type="text/xsl" href="configuration.xsl"?>
<!--
  Licensed under the Apache License, Version 2.0 (the "License");
  you may not use this file except in compliance with the License.
  You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

  Unless required by applicable law or agreed to in writing, software
  distributed under the License is distributed on an "AS IS" BASIS,
  WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
  See the License for the specific language governing permissions and
  limitations under the License. See accompanying LICENSE file.
-->

<!-- Put site-specific property overrides in this file. -->

<configuration>
<property>
    <name>dfs.replication</name>
    <value>1</value>
  </property>

</configuration>
EOF

In [4]:
%%bash
cat <<EOF > $HADOOP_HOME/etc/hadoop/mapred-site.xml
<?xml version="1.0"?>
<?xml-stylesheet type="text/xsl" href="configuration.xsl"?>
<!--
  Licensed under the Apache License, Version 2.0 (the "License");
  you may not use this file except in compliance with the License.
  You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

  Unless required by applicable law or agreed to in writing, software
  distributed under the License is distributed on an "AS IS" BASIS,
  WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
  See the License for the specific language governing permissions and
  limitations under the License. See accompanying LICENSE file.
-->

<!-- Put site-specific property overrides in this file. -->

<configuration>
<property>
    <name>mapreduce.framework.name</name>
    <value>yarn</value>
  </property>
  <property>
    <name>mapreduce.application.classpath</name>
    <value>$HADOOP_HOME/share/hadoop/mapreduce/*:$HADOOP_HOME/share/hadoop/mapreduce/lib/*</value>
  </property>

</configuration>
EOF

In [5]:
%%bash
cat <<EOF > $HADOOP_HOME/etc/hadoop/yarn-site.xml
<?xml version="1.0"?>
<!--
  Licensed under the Apache License, Version 2.0 (the "License");
  you may not use this file except in compliance with the License.
  You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

  Unless required by applicable law or agreed to in writing, software
  distributed under the License is distributed on an "AS IS" BASIS,
  WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
  See the License for the specific language governing permissions and
  limitations under the License. See accompanying LICENSE file.
-->
<configuration>
<property>
    <description>The hostname of the RM.</description>
    <name>yarn.resourcemanager.hostname</name>
    <value>localhost</value>
  </property>
  <property>
    <name>yarn.nodemanager.aux-services</name>
    <value>mapreduce_shuffle</value>
  </property>
  <property>
    <name>yarn.nodemanager.env-whitelist</name>
    <value>JAVA_HOME,HADOOP_COMMON_HOME,HADOOP_HDFS_HOME,HADOOP_CONF_DIR,CLASSPATH_PREPEND_DISTCACHE,HADOOP_YARN_HOME,HADOOP_HOME,PATH,LANG,TZ,HADOOP_MAPRED_HOME</value>
  </property>

<!-- Site specific YARN configuration properties -->

</configuration>
EOF

# Konfigurasi File HDFS

Sebelum HDFS dapat digunakan, sistem file harus dikonfigurasi. Proses konfigurasi membuat sistem file kosong dengan membuat direktori penyimpanan dan versi awal NameNodes

In [6]:
!$HADOOP_HOME/bin/hdfs namenode -format

#Creating other necessary enviroment variables before starting nodes
os.environ["HDFS_NAMENODE_USER"] = "root"
os.environ["HDFS_DATANODE_USER"] = "root"
os.environ["HDFS_SECONDARYNAMENODE_USER"] = "root"
os.environ["YARN_RESOURCEMANAGER_USER"] = "root"
os.environ["YARN_NODEMANAGER_USER"] = "root"

#Launching hdfs deamons
!$HADOOP_HOME/sbin/start-dfs.sh

#Launching yarn deamons
#nohup causes a process to ignore a SIGHUP signal
!nohup $HADOOP_HOME/sbin/start-yarn.sh

#Listing the running deamons
!jps

2024-05-04 12:56:35,416 INFO namenode.NameNode: STARTUP_MSG: 
/************************************************************
STARTUP_MSG: Starting NameNode
STARTUP_MSG:   host = 7c8cea03b113/172.28.0.12
STARTUP_MSG:   args = [-format]
STARTUP_MSG:   version = 3.2.3
STARTUP_MSG:   classpath = /usr/local/hadoop-3.2.3/etc/hadoop:/usr/local/hadoop-3.2.3/share/hadoop/common/lib/jsr305-3.0.2.jar:/usr/local/hadoop-3.2.3/share/hadoop/common/lib/commons-net-3.6.jar:/usr/local/hadoop-3.2.3/share/hadoop/common/lib/javax.servlet-api-3.1.0.jar:/usr/local/hadoop-3.2.3/share/hadoop/common/lib/kerb-simplekdc-1.0.1.jar:/usr/local/hadoop-3.2.3/share/hadoop/common/lib/jackson-core-asl-1.9.13.jar:/usr/local/hadoop-3.2.3/share/hadoop/common/lib/metrics-core-3.2.4.jar:/usr/local/hadoop-3.2.3/share/hadoop/common/lib/jaxb-api-2.2.11.jar:/usr/local/hadoop-3.2.3/share/hadoop/common/lib/kerby-config-1.0.1.jar:/usr/local/hadoop-3.2.3/share/hadoop/common/lib/commons-compress-1.21.jar:/usr/local/hadoop-3.2.3/share/h

In [7]:
#Report the basic file system information and statistics
!$HADOOP_HOME/bin/hdfs dfsadmin -report

Configured Capacity: 115658190848 (107.72 GB)
Present Capacity: 85084016640 (79.24 GB)
DFS Remaining: 85083992064 (79.24 GB)
DFS Used: 24576 (24 KB)
DFS Used%: 0.00%
Replicated Blocks:
	Under replicated blocks: 0
	Blocks with corrupt replicas: 0
	Missing blocks: 0
	Missing blocks (with replication factor 1): 0
	Low redundancy blocks with highest priority to recover: 0
	Pending deletion blocks: 0
Erasure Coded Block Groups: 
	Low redundancy block groups: 0
	Block groups with corrupt internal blocks: 0
	Missing block groups: 0
	Low redundancy blocks with highest priority to recover: 0
	Pending deletion blocks: 0

-------------------------------------------------
Live datanodes (1):

Name: 127.0.0.1:9866 (localhost)
Hostname: 7c8cea03b113
Decommission Status : Normal
Configured Capacity: 115658190848 (107.72 GB)
DFS Used: 24576 (24 KB)
Non DFS Used: 30557396992 (28.46 GB)
DFS Remaining: 85083992064 (79.24 GB)
DFS Used%: 0.00%
DFS Remaining%: 73.57%
Configured Cache Capacity: 0 (0 B)
Cache

# Map Reduce Menggunakan Python



In [8]:
#Exploring Hadoop utilities available
!ls $HADOOP_HOME/share/hadoop/tools/lib/

aliyun-java-sdk-core-4.5.10.jar      hadoop-gridmix-3.2.3.jar
aliyun-java-sdk-kms-2.11.0.jar	     hadoop-kafka-3.2.3.jar
aliyun-java-sdk-ram-3.1.0.jar	     hadoop-openstack-3.2.3.jar
aliyun-sdk-oss-3.13.0.jar	     hadoop-resourceestimator-3.2.3.jar
aws-java-sdk-bundle-1.11.901.jar     hadoop-rumen-3.2.3.jar
azure-data-lake-store-sdk-2.2.9.jar  hadoop-sls-3.2.3.jar
azure-keyvault-core-1.0.0.jar	     hadoop-streaming-3.2.3.jar
azure-storage-7.0.0.jar		     ini4j-0.5.4.jar
hadoop-aliyun-3.2.3.jar		     jdom2-2.0.6.jar
hadoop-archive-logs-3.2.3.jar	     kafka-clients-2.8.1.jar
hadoop-archives-3.2.3.jar	     lz4-java-1.7.1.jar
hadoop-aws-3.2.3.jar		     ojalgo-43.0.jar
hadoop-azure-3.2.3.jar		     opentracing-api-0.33.0.jar
hadoop-azure-datalake-3.2.3.jar      opentracing-noop-0.33.0.jar
hadoop-datajoin-3.2.3.jar	     opentracing-util-0.33.0.jar
hadoop-distcp-3.2.3.jar		     org.jacoco.agent-0.8.5-runtime.jar
hadoop-extras-3.2.3.jar		     wildfly-openssl-1.0.7.Final.jar
hadoop-fs2img-3.2.3.

In [9]:
#Dowloading text example to use as input
!wget -q https://www.gutenberg.org/cache/epub/73525/pg73525.txt

In [10]:
#Creating directory in HDFS
!$HADOOP_HOME/bin/hdfs dfs -mkdir /zdn_hw

In [11]:
#Copying the file from local file system to Hadoop distributed file system (HDFS)
!$HADOOP_HOME/bin/hdfs dfs -put /content/pg73525.txt /zdn_hw

## Mapper

Mapper adalah komponen dalam MapReduce yang memproses dan mengelompokkan data input menjadi pasangan kunci-nilai.

In [12]:
%%writefile mapper.py
#!/usr/bin/env python

import sys

# Mapper membaca setiap baris dari STDIN
for line in sys.stdin:
    line = line.strip()  # Menghapus spasi di awal dan akhir
    if 'me' in line:  # Periksa jika kata 'me' ada dalam baris
        print('%s\t%s' % (line, 1))


Writing mapper.py


#Reducer
Reducer adalah komponen dalam MapReduce yang mengolah pasangan kunci-nilai dari Mapper untuk menghasilkan output yang teragregasi atau diringkas.

In [13]:
%%writefile reducer.py
#!/usr/bin/env python

import sys

current_sentence = None
sentence_count = 0
total_sentences_with_me = 0

# Membaca input dari STDIN
for line in sys.stdin:
    line = line.strip()
    sentence, count = line.split('\t', 1)

    # Mengonversi count menjadi integer
    try:
        count = int(count)
    except ValueError:
        continue

    # Jika kalimat sama dengan kalimat saat ini, tambahkan count
    if current_sentence == sentence:
        sentence_count += count
    else:
        if current_sentence:
            # Cetak hanya kalimat tanpa jumlah
            print('Kalimat: "%s"' % current_sentence)
            total_sentences_with_me += sentence_count
        current_sentence = sentence
        sentence_count = count

# Jangan lupa untuk mencetak kalimat terakhir jika ada
if current_sentence:
    print('Kalimat: "%s"' % current_sentence)
    total_sentences_with_me += sentence_count

# Mencetak total kalimat yang mengandung 'me'
print('Total kalimat yang mengandung "me": %s' % total_sentences_with_me)


Writing reducer.py


In [14]:
#Testing our MapReduce job locally (Hadoop does not participate here)
!cat pg73525.txt | python mapper.py | sort -k1,1 | python reducer.py | head -50
#We apply sorting after the mapper because it is the default operation in MapReduce architecture

Kalimat: "• You comply with all other terms of this agreement for free"
Kalimat: "1.F.4. Except for the limited right of replacement or refund set forth"
Kalimat: "1.F.5. Some states do not allow disclaimers of certain implied"
Kalimat: "accordance with this agreement, and any volunteers associated with the"
Kalimat: "active links or immediate access to the full terms of the Project"
Kalimat: "agree to be bound by the terms of this agreement. There are a few"
Kalimat: "agreement and help preserve free future access to Project Gutenberg™"
Kalimat: "agreement before downloading, copying, displaying, performing,"
Kalimat: "agreement shall be interpreted to make the maximum disclaimer or"
Kalimat: "airmen are made need not know anything--or much. See the point? You’re"
Kalimat: "all set to fly alone. Then, that same day, they took the 23 away. The"
Kalimat: "and came out in language that lay people erroneously suppose is solely"
Kalimat: "and robs me but my ex-cook, that rotten cook, Shane

In [15]:
#Changing the permissions of the files
!chmod 777 /content/mapper.py /content/reducer.py
#Setting 777 permissions to a file or directory means that it will be readable, writable and executable by all users

In [16]:

#Running MapReduce programs
!$HADOOP_HOME/bin/hadoop jar $HADOOP_HOME/share/hadoop/tools/lib/hadoop-streaming-3.2.3.jar \
  -input /zdn_hw/pg73525.txt \
  -output /zdn_hw/output \
  -mapper "python /content/mapper.py" \
  -reducer "python /content/reducer.py"

packageJobJar: [/tmp/hadoop-unjar241783165532311732/] [] /tmp/streamjob4092437308045423772.jar tmpDir=null
2024-05-04 12:58:01,758 INFO client.RMProxy: Connecting to ResourceManager at localhost/127.0.0.1:8032
2024-05-04 12:58:02,067 INFO client.RMProxy: Connecting to ResourceManager at localhost/127.0.0.1:8032
2024-05-04 12:58:02,446 INFO mapreduce.JobResourceUploader: Disabling Erasure Coding for path: /tmp/hadoop-yarn/staging/root/.staging/job_1714827430539_0001
2024-05-04 12:58:02,829 INFO mapred.FileInputFormat: Total input files to process : 1
2024-05-04 12:58:03,764 INFO mapreduce.JobSubmitter: number of splits:2
2024-05-04 12:58:04,004 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1714827430539_0001
2024-05-04 12:58:04,006 INFO mapreduce.JobSubmitter: Executing with tokens: []
2024-05-04 12:58:04,308 INFO conf.Configuration: resource-types.xml not found
2024-05-04 12:58:04,308 INFO resource.ResourceUtils: Unable to find 'resource-types.xml'.
2024-05-04 12:58:04,98

In [17]:
#Exploring the created output directory
#part-r-00000 contains the actual ouput
!$HADOOP_HOME/bin/hdfs dfs -ls /zdn_hw/output

Found 2 items
-rw-r--r--   1 root supergroup          0 2024-05-04 12:58 /zdn_hw/output/_SUCCESS
-rw-r--r--   1 root supergroup      10074 2024-05-04 12:58 /zdn_hw/output/part-00000


In [18]:
#Printing out first 50 lines
!$HADOOP_HOME/bin/hdfs dfs -cat /zdn_hw/output/part-00000 | head -50

Kalimat: "(trademark/copyright) agreement. If you do not agree to abide by all"	
Kalimat: "1.F.4. Except for the limited right of replacement or refund set forth"	
Kalimat: "1.F.5. Some states do not allow disclaimers of certain implied"	
Kalimat: "Any number of ex-overseas men could answer this."	
Kalimat: "Black. Had myself nicely planted in the rear seat, and who comes out"	
Kalimat: "Bob Watts was flying it this morning and he had the same trouble.”"	
Kalimat: "But the sergeant turned out to be a handy mechanic. Good mechanics were"	
Kalimat: "Gutenberg Literary Archive Foundation. Royalty payments must be paid"	
Kalimat: "Gutenberg™ electronic work under this agreement, disclaim all"	
Kalimat: "M.P’s apart at high noon of the same day in the public square at"	
Kalimat: "Master Sergeant Sciples, in charge of the hangar, came along to start"	
Kalimat: "New classes of cadets came to that field. Even one of the cooks from"	
Kalimat: "New made flying cadets came to that field. Lord! Wh

In [19]:
!$HADOOP_HOME/bin/hdfs dfs -copyToLocal /zdn_hw/output/part-00000 /content/jumlah_kalimat_mengandung_me.txt